In [27]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast
from tqdm import tqdm


with open('../DATASET/Odia_Dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert the dictionary to a list of tuples
dataset = list(data.items())

# Load the RoBERTa base model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Load the training data
train_data = load_dataset('glue', 'mnli')

# Preprocess the training data
def preprocess_function(example):
    return tokenizer(example['premise'], example['hypothesis'], padding='max_length', truncation=True, max_length=512)

train_dataloader = DataLoader(train_data['train'].map(preprocess_function), batch_size=16)

# Define the training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(5):
    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate the performance of the model on the validation set
validation_data = load_dataset('glue', 'mnli')

def evaluate_function(example):
    return tokenizer(example['premise'], example['hypothesis'], padding='max_length', truncation=True, max_length=512)

validation_dataloader = DataLoader(validation_data['validation'].map(evaluate_function), batch_size=16)

correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(validation_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += len(labels)

accuracy = correct / total
print(f"Accuracy: {accuracy}")

# Save the trained model
torch.save(model.state_dict(), 'roberta-trained.pt')

    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should pr

NameError: name 'load_dataset' is not defined